In [1]:
!pip install jageocoder
!pip install javascript
!pip install googlemaps
!pip install OSMPythonTools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=edf4c3efbb53153c627b21b38636df00b5bce35adb5efb69783eabc439019e9a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for jaconv: filename=jaconv-0.3.4-py3-none-any.whl size=16416 sha256=742019c018421381080dedd09537850bb394dcd1b2a14acd33eb5132c10ce33d
  Stored in directory: /root/.cache/pip/wheels/46/8f/2e/a730bf1fca05b33e532d5d91dabdf406c9b

In [2]:
!wget https://www.info-proto.com/static/jageocoder/latest/v2/jukyo_all_v20.zip
!jageocoder install-dictionary jukyo_all_v20.zip

--2023-06-21 03:23:24--  https://www.info-proto.com/static/jageocoder/latest/v2/jukyo_all_v20.zip
Resolving www.info-proto.com (www.info-proto.com)... 118.27.100.213
Connecting to www.info-proto.com (www.info-proto.com)|118.27.100.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 786353338 (750M) [application/zip]
Saving to: ‘jukyo_all_v20.zip’

jukyo_all_v20.zip   100%[===================>] 749.92M  25.9MB/s    in 30s     

2023-06-21 03:23:55 (25.2 MB/s) - ‘jukyo_all_v20.zip’ saved [786353338/786353338]

INFO:jageocoder.module:210:Please read '/usr/jageocoder/db2/README.md' for terms and conditions of use.
INFO:jageocoder.module:214:Installation completed.


In [3]:
import json
import pprint
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Optional, Dict, Tuple, Union
import requests
import jageocoder
from OSMPythonTools.nominatim import Nominatim
import googlemaps
from javascript import require

In [11]:
YOUR_GOOGLE_CLIENT_KEY = "<YOUR GCP CLIENT KEY>"

In [5]:
js_JSON = require("JSON")
geolonia = require('@geolonia/normalize-japanese-addresses')

 Installing 'JSON' version 'latest'... This will only happen once. 

[JSE] npm notice created a lockfile as package-lock.json. You should commit this file.
[JSE] npm WARN js-modules@ No repository field.
[JSE] npm WARN js-modules@ No license field.
added 1 package from 2 contributors and audited 1 package in 0.828s

found 0 vulnerabilities





 OK. 

 Installing '@geolonia/normalize-japanese-addresses' version 'latest'... This will only happen once. 

[JSE] npm WARN js-modules@ No repository field.
[JSE] npm WARN js-modules@ No license field.
added 10 packages from 7 contributors and audited 11 packages in 5.827s

found 0 vulnerabilities





 OK. 



In [6]:
GEOCODING_API_URLS = {
    "IMI": "https://imi-enrichment-address.vercel.app/api",
    "GSI": "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
}


nominatim = Nominatim()
jageocoder.init()

def postalcode_formatter(note):
    if not note:
        return ""
    results = note.split("/")
    results = dict([r.split(":") for r in results])
    if "postcode" not in results:
        return ""
    postal_code = ""
    for k, v in results.items():
        if k.strip() == "postcode":
            postal_code = v.strip()
    return f"〒{postal_code[:3]}-{postal_code[3:]}"

gmaps = googlemaps.Client(key=YOUR_GOOGLE_CLIENT_KEY)

In [7]:
@dataclass
class Address:
    title: str
    postal_code: Optional[str] = None
    pref: Optional[str] = None
    city: Optional[str] = None
    town: Optional[str] = None
    addr: Optional[str] = None
    coords: Optional[Tuple[float, float]] = None
    raw: Dict[str, Union[Optional[str], Optional[float], int]] = field(default_factory=dict)


def geolonia_search():
    def search(address):
        result = json.loads(js_JSON.stringify(geolonia.normalize(address, { "level": 3 })))
        address = Address(
                title="".join(result[k] for k in ["pref", "city", "town", "addr"]),
                pref=result["pref"],
                city=result["city"],
                town=result["town"],
                addr=result["addr"],
                coords=(result["lat"], result["lng"]),
                raw=result)
        return [address]
    return search

def imi_search():
    def search(address):
        res = requests.post(GEOCODING_API_URLS["IMI"], json={"@type": "住所型", "表記": address})
        result = res.json()
        normalized_result = []
        for k in ["都道府県", "市区町村", "区", "町名", "丁目", "番地", "号"]:
            if k in result:
                if k in ["丁目", "番地", "号"]:
                    target = result[k] + k
                else:
                    target = result[k]
                normalized_result.append(target)
        address = Address(
            title="".join(normalized_result),
            pref=result["都道府県"],
            city=result["市区町村"] + result.get("区", ""),
            town=result.get("町名"),
            addr="".join(normalized_result[-3:]),
            raw=result,
        )
        return [address]
    return search

def jageocoder_search():
    def search(address):
        result = jageocoder.search(address)

        candidates = []
        for cand in result["candidates"]:
            postal_code = postalcode_formatter(cand["note"]).rstrip("〒")
            title = f"{postal_code} {''.join(cand['fullname'])}"
            address = Address(
                title=title,
                postal_code=postal_code if postal_code != "" else None,
                pref=cand["fullname"][0],
                city=cand["fullname"][1],
                town=cand["fullname"][2],
                addr="".join(cand["fullname"][3:]),
                coords=(cand["y"], cand["x"]),
                raw=cand,
            )
            candidates.append(address)
        return candidates
    return search

def gsi_search():
    def search(address):
        res = requests.get(GEOCODING_API_URLS["GSI"] + address)
        results = res.json()

        candidates = []
        for result in results:
            address = Address(
                title=result["properties"]["title"],
                coords=(result["geometry"]["coordinates"][1], result["geometry"]["coordinates"][0]),
                raw=result,
            )
            candidates.append(address)
        return candidates
    return search

def nominatim_search():
    def search(address):
        location = nominatim.query(address)
        candidates = []
        for loc in location._json:
            names = list(reversed(loc["display_name"].split(", ")))
            result_name = f"{names[0]}、〒{names[1]} {''.join(names[2:])}"
            address = Address(
                title=result_name,
                postal_code=names[1],
                pref=names[2],
                city=names[3],
                town=names[4],
                addr="".join(names[5:]),
                coords=(loc["lat"], loc["lon"])
            )
            candidates.append(address)
        return candidates
    return search

def gmap_search():
    def search(address):
        results = gmaps.places(address, language="ja")["results"]
        candidates = []

        for result in results:
            title = result.get("formatted_address", "").strip()
            address_text= title.strip("、")[1]
            postal_code = address_text.strip(" ")[0]

            address = Address(
                title=title,
                postal_code=postal_code.rstrip("〒"),
                coords=(result["geometry"]["location"]["lat"], result["geometry"]["location"]["lng"]),
                raw=result,
            )
            candidates.append(address)
        return candidates
    return search

In [8]:
ADDRESSES = [
    "新潟県新潟市北区東栄町",
    "千葉県浦安市舞浜2-11",
    "千葉県浦安市舞浜2番地11号",
    "千葉県浦安市舞浜2丁目11",
    "静岡県下田市2丁目4-26",
    "埼玉県春日部市八丁目353",
    "奈良県御所市1番地の3",
    "大阪市中央区上町A-12",
    "大阪市中央区久太郎町四丁目渡辺",
    "千葉県香取市佐原イ",
    "千葉県香取市佐原口", # 入力ミスに対応できるか
    "長野県長野市南長野県町",
]

In [9]:
ANALYZERS = {
    "Google Places API": gmap_search(),
    "Nominatim": nominatim_search(),
    "jageocoder": jageocoder_search(),
    "国土地理院": gsi_search(),
    "IMI": imi_search(),
    "@geolonia/normalize-japanese-addresses": geolonia_search(),
}

In [10]:
pp = pprint.PrettyPrinter(indent=2)
normalized_results = defaultdict(dict)

for address in ADDRESSES:
    for key, analyzer in ANALYZERS.items():
        result = analyzer(address)
        if result:
            normalized_results[address][key] = result
        else:
            normalized_results[address][key] = []

for target, results in normalized_results.items():
    print("【" + target + "】")
    for service, result in results.items():
        print(service + ":")
        pp.pprint(result)
        print("\n")
    print("----------------------------------")

【新潟県新潟市北区東栄町】
Google Places API:
[ Address(title='日本、〒950-3104 新潟県新潟市北区東栄町',
          postal_code='本',
          pref=None,
          city=None,
          town=None,
          addr=None,
          coords=(37.9539941, 139.1867389),
          raw={ 'formatted_address': '日本、〒950-3104 新潟県新潟市北区東栄町',
                'geometry': { 'location': { 'lat': 37.9539941,
                                            'lng': 139.1867389},
                              'viewport': { 'northeast': { 'lat': 37.95495810000001,
                                                           'lng': 139.1905448},
                                            'southwest': { 'lat': 37.9502019,
                                                           'lng': 139.1792868}}},
                'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
                'icon_background_color': '#7B9EB0',
                'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gene